# Содержание
* [Определение стоимости автомобилей](#section_id5)</a>
* [1. Подготовка данных](#section_id1)</a>
* [2. Обучение модели](#section_id2)</a>
* [3. Анализ моделей](#section_id3)</a>
* [Вывод](#section_id4)</a>
* [4. Чек-лист готовности проекта](#section_id6)</a>

# Определение стоимости автомобилей
<a id='section_id5'></a>

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных
<a id='section_id1'></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
import time
import lightgbm as lgb
import os


In [ ]:
# Простые модели, на которых будем проверять
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Вспомогательные методы
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import OrdinalEncoder

from sklearn.compose import make_column_transformer

In [ ]:
# буст
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor


In [ ]:
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
DATA_SHAPE = df.shape[0]
DATA_SHAPE

* Признаки
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя


* Целевой признак
* Price — цена (евро)

In [ ]:
df #Выведим таблицу для ознокомления с ней. 

In [ ]:
df.info()

In [ ]:
df.duplicated().sum() # Проверим есть ли явные дублекаты

In [ ]:
df = df.drop_duplicates().reset_index(drop= True)# удалите дубликаты и приведем индексы в порядок

In [ ]:
#Удаляем неинформативные признаки

df= df.drop(['DateCrawled','DateCreated','LastSeen','NumberOfPictures', 'PostalCode'], axis=1)


In [ ]:
#Приведем названия для дальнейшей работы к змеинному иедексу
df.columns = df.columns = ['Price', 'Vehicle_Type', 'Registration_Year', 'Gearbox', 'Power',
       'Model', 'Kilometer', 'Registration_Month', 'Fuel_Type',
       'Brand', 'Repaired']
df.columns = map(str.lower, df.columns) #Нестоит забывать про нижний регистр

In [ ]:
df.isna().sum() # Посмотрим пропуски.

In [ ]:
df.describe()

Сразу бросаются в глаза явные перекосы, такие как неодекваный год. Необходимо привести данные впорядок.

Сначала посмотрим на количество явно ошибочных данных

In [ ]:
print('Машины после 2023 года: ', df.query('registration_year > 2023')['registration_year'].count())# Датасет скорее всего до 2023 года

In [ ]:
df['model'].value_counts() #для определения года выпуска можно былобы посмотреть начала производства модели но 250 слишком долго.

In [ ]:
print('Машины до 1913 года: ', df.query('registration_year < 1913')['registration_year'].count())# 1913 год 1 конвеера для машин

In [ ]:
print('Машины 2023 года: ',df[df['registration_year'] == 2023]['registration_year'].count())
print('Машины 2022 года: ',df[df['registration_year'] == 2022]['registration_year'].count())
print('Машины 2021 года: ',df[df['registration_year'] == 2021]['registration_year'].count())
print('Машины 2020 года: ',df[df['registration_year'] == 2020]['registration_year'].count())
print('Машины 2019 года: ',df[df['registration_year'] == 2019]['registration_year'].count())
#Значит датасет заканчивается машинами 2019 года регестрации

In [ ]:
df.drop(index=df.query('registration_year > 2023').index,inplace=True)# 2024 год еще ненаступил

In [ ]:
file_path = '/datasets/autos.csv'
stat_info = os.stat(file_path)
file_modified_time = pd.Timestamp(stat_info.st_mtime, unit='s')
print(file_modified_time)
print(file_modified_time.min)
print(file_modified_time.max)

In [ ]:
df.drop(index=df.query('registration_year < 1913').index,inplace=True)# Невижу смысла расматривать машины сделанные раньше

In [ ]:
print('Машины до 1939 года: ', df.query('registration_year < 1939')['registration_year'].count())
# Может старые машины будут интересны колекционерам. Посмотрим более подробно выпуск до 2 мировой войны

In [ ]:
df[df['registration_year'] < 1939]

Почти все модели неизвестны. Мне удалось найти kadett и 500 (похоже на Fiat 500) эти машины выпускались с 1936 года.Особой кореляции по цене тут невидно и проверить другие марки нет возможности. Поэтому можем смело удалить машины до 1936 года. А вот citroen c3 вро де как 2012 года, скорее всего опечатались с датой.	

In [ ]:
df.drop(index=df.query('registration_year < 1936').index,inplace=True)

In [ ]:
df['registration_month'].value_counts()

У нас появился 0 месяц которого в календаре нет. Мы пока можем расмотреть 0 месяц как пропуск. Таких пропусков довольно много. Месяц продажи мог бы говарить нам о навизне машины, но поскольку 2019 год 25 машин, да и периуд у нас довольно большой, думаю этот параметр был бы полезен если бы мы расматривали машины последних 3 максимум 5 лет. Более полезен этот пораметр брать если расматривать сезонность, но недумаю, что сезон имеет сильное значение для цены автомобиля. Лучше посмотрим кореляцию и решим стоит ли нам использовать данное значение для обучения.

In [ ]:
list_for_graph = ['registration_year',
                  'power',
                  'kilometer',
                  'registration_month']

for entry in list_for_graph:
    correlation = df['price'].corr(df[entry])
    print(f'Корреляция между price и {entry} составляет: {correlation:0.1%}')

Оно является самым маленьким значением, если удалить 0 месяц то падает еще ниже. Поэтому смело удаляем как неинформативный признак.

Смощьностью сложнее. Если мощьность 0 это может означать как отсутствие информации так и то, что мотор отсутствует или неработает. Также малая мощьность может говарить о неверных значениях или очень старом двигателе. Очень высокая об ошибке или штучном суперкаре. 

In [ ]:
print('Машины более 560 лошадиных сил: ', df.query('power > 560')['power'].count())

In [ ]:
df.drop(index=df.query('power > 560').index,inplace=True)

In [ ]:
df[df['power'] > 500].head(30)

Некоторые машины такие как twingo или polo их марки не превышают 150 лошадинных сил, clk более мощьные но меньше 400 лошадей другие как mustang могут иметь более 500 лошадок, а недорогие как golf имеют меньшую мощьность. Вобщем если удалить до 500 евро это решит проблему с дешовым и маломощьным сегментом, но всеровно будет много ошибок. Я думая если удалим более мощьные двигатели чем максимумальный clk это 388 лошадиных сил, то увидем меньше ошибок.

In [ ]:
print('Машины более 388 лошадиных сил: ', df.query('power > 388')['power'].count())

In [ ]:
df.drop(index=df.query('power > 388').index,inplace=True)

Расмотрим с 0 значением

In [ ]:
print('Машины с 0 лошадиных сил: ', df.query('power == 0')['power'].count())

In [ ]:
df[df['power'] == 0].head(40)

Вообще похоже на то, что присутствует 3 варианта. Кто то продает машину явно со сломанным двигателем поскольку для машины на ходу очень дешево, кто то толи незнал толи по ошибки постваил 0. Поскольку мы будем обучать модель и предсказывать цену нас будут путать сломанные машины поскольку они не являются отдельной категорией поэтому нам лучше востановить мощьность двигателя по такимже автомобилям. 

In [ ]:
def calculate_power(df, brand, model, year):
    df_part = df[(df['registration_year'] == year) & (df['brand'] == brand) & (df['model'] == model)]
    if (df_part['power'].count() == 0):
        # если не нашли по году, бренду и модели, пробуем только по бренду + модели
        df_part = df[(df['brand'] == brand) & (df['model'] == model)]
    
    median_power = df_part['power'].median()
    
    if (math.isnan(median_power)):
        median_power = 0
        
    return median_power

# Попробуем заполнить мощность для наших плохих данных. Рамки возьмем как Fiat 126 с 26 л.с. и доработаный Fiat 500 с 300 л.с.
bad_power_df = df[(df['power'] < 26) | (df['power'] > 300)][['brand', 'model', 'registration_year']]
bad_power_df.count()

In [ ]:
# Заполним незаполненные модели категорией other
bad_power_df['model'] = bad_power_df['model'].fillna('other')

In [ ]:
bad_power_df = bad_power_df.drop_duplicates()
bad_power_df.count()

In [ ]:
%%time

for row in bad_power_df.itertuples(index=True, name='pandas'):
    year = getattr(row, "registration_year")
    brand = getattr(row, "brand")
    model = getattr(row, "model")
    median = calculate_power(df, brand, model, year)
    searchRow = ((df['power'] < 30) | (df['power'] > 340)) & (df['brand'] == brand) \
    & (df['model'] == model) & (df['registration_year'] == year)
    df.loc[searchRow, 'power'] = median

In [ ]:
# Посмотрим, сколько нулевых значений осталось
df[df['power'] == 0]['price'].count()

In [ ]:
df['power'] = df['power'].astype(int)

In [ ]:
# Лучше удалим оставшиеся значения = 0
df.drop(index=df.query('power == 0').index,inplace=True)

С ценой возникает чем то схожая проблема. Если мы оставим 0 значения и очень дешовые то собьем выборку поскольку у нас нет четкого пониманиярабочая машина или нет.

In [ ]:
print('Машиныстоимостью до 500 евра: ', df.query('price < 500')['price'].count())

In [ ]:
# Врядли машины дешевле 500 евро на ходу. Чтобы при обучении непортить модель сделаем похожие монипуляции как и с мощьностью.
# Общую цену лучше взять по средней

def calculate_price(df, brand, model, year):
    # в среднем не будем рассчитывать нулевые значения
    df_part = df[(df['price'] > 500) & (df['registration_year'] == year) & (df['brand'] == brand) & (df['model'] == model)]
    if (df_part['price'].count() == 0):
        # если не нашли по году, бренду и модели, пробуем только по бренду + модели
        df_part = df[(df['price'] > 500) & (df['brand'] == brand) & (df['model'] == model)]
    
    mean_price = df_part['price'].mean()
    
    if (math.isnan(mean_price)):
        mean_price = 0
        
    return mean_price

# Попробуем заполнить мощность для наших плохих данных
bad_price_df = df[df['price'] < 500][['brand', 'model', 'registration_year']]
bad_price_df['model'] = bad_price_df['model'].fillna('other')
bad_price_df = bad_price_df.drop_duplicates()
bad_price_df.count()

In [ ]:
%%time

for row in bad_price_df.itertuples(index=True, name='pandas'):
    year = getattr(row, "registration_year")
    brand = getattr(row, "brand")
    model = getattr(row, "model")
    mean_price = calculate_price(df, brand, model, year)
    searchRow = (df['price'] < 500) & (df['brand'] == brand) & (df['model'] == model) & (df['registration_year'] == year)
    df.loc[searchRow, 'price'] = mean_price
    
df[df['price'] < 500]['price'].count()

In [ ]:
print('Машины стоимостью до 500 евра: ', df.query('price < 500')['price'].count())

In [ ]:
df['price'] = df['price'].astype(int)

In [ ]:
df.drop(index=df.query('price < 500').index,inplace=True)

Перейдем к признакам в которых так сильно ошибки небрасались.

In [ ]:
# Посмотрим, есть ли артифакты в значениях repaired
df['repaired'].value_counts()

In [ ]:
# Переведем да, нет и пропуск в цивры. Так мы избавимся от пропусков и нам будет легче использовать эти данные.
# 0 будет отсутствие ремонта, 1 неизвестно и 2 - ремонт был (таким образом определим вес каждой категории)

df['repaired'] = df['repaired'].fillna(1)

def fill_not_repaired(value):
    if value == 'no':
        return 0
    elif value == 'yes':
        return 2
    else:
        return value

df['repaired'] = df['repaired'].apply(fill_not_repaired)
df['repaired'].value_counts()

In [ ]:
df['repaired'] = df['repaired'].astype(int)

In [ ]:
# Расмотрим vehicle_type
df['vehicle_type'].value_counts()

In [ ]:
# Делаем отдельную категорию
df['vehicle_type'] = df['vehicle_type'].fillna('undefined')

In [ ]:
# Убираем пропуски заменив их новой категорией

df['gearbox'] = df['gearbox'].fillna('undefined')
df['model'] = df['model'].fillna('undefined')
df['fuel_type'] = df['fuel_type'].fillna('undefined')
df['brand'] = df['brand'].fillna('undefined')

In [ ]:
df['fuel_type'].value_counts()

In [ ]:
df['fuel_type'].replace({'petrol': 'gasoline'}, inplace=True)

In [ ]:
df['fuel_type'].value_counts()

In [ ]:
df.reset_index(drop=True,inplace=True) #Cбросим индексацию после удаления строк.

In [ ]:
DATA_SHAPE = df.shape[0] / DATA_SHAPE
DATA_SHAPE

1. Резюме.

* Загрузил данные и изучил данные. 
* Проверим есть ли явные дублекаты. Их оказалось 4, удалили их.
* Удаляем неинформативные признаки 'DateCrawled','DateCreated','LastSeen','NumberOfPictures', 'PostalCode'.
* Приведем названия для дальнейшей работы к змеинному иедексу.
* Проверяем на аномалии. Их бросается в глаза довольно много, особенно год выпуска и мощьность автомобиля. Также настораживает цена. Поскольку это целевой признак мы к нему должны относится особо бережно. Год мы берем с 1913 первый автомобильный канвеер по 2023 (текущий). Проверили месяц на кореляцию с ценой, результат вышел небольшим, удалили этот признак как и другие неинформативные признаки. Было решено удалить машины с мощьностью более 388 л.с. и был  использвать перебор для машин мощьностью менее 26 л.с. им присвоины мощьности соответствующие авто с совпадающим годом моделью и маркой автомобиля.  Конечно 0 мощьность мотора может значит, что машина не находу, но тогда другие признаки сильно теряют свою актуальность. С ценой было проделана аналогичная операция с диапозоном менее 500 евра. Машины на которые неподошли похожие были удалены.
* Данные по присутствию или отсутствию ремонта мы перевели в цифровой вид  0 будет отсутствие ремонта, 1 неизвестно и 2 - ремонт был .
* Остальные пропуски заменили категорией undefined.
* Признаки которые могли перевели в int.
* Было удалено менее 4% информации с датасета.
* Преравнял "Petrol" и "gasoline" друг к другу.

## Обучение моделей
<a id='section_id2'></a>

Подготовка выборок для обучения моделей

<div class="alert alert-block alert-warning">
<b>1.Изменения:</b> Были внесены следующие изменения: Перенес сюда данный метод кодирования, другой метод закаментировал, это должно исправить большинство ошибок.
</div>

In [ ]:
# # помним про PEP-8
# # импорты из стандартной библиотеки
# import warnings

# # импорты сторонних библиотек
# import numpy as np
# import pandas as pd

# # импорты модулей текущего проекта
# # длина строки до 78 символов
# from sklearn.compose import make_column_transformer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import Ridge
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import (
#     GridSearchCV, 
#     RandomizedSearchCV,
#     train_test_split
# )
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import (
#     OneHotEncoder,
#     OrdinalEncoder,
#     StandardScaler
# )

# # настройки
# warnings.filterwarnings("ignore")

# # константы заглавными буквами
# RANDOM_STATE = 42

In [ ]:
# def get_data_info(data):
#     display(data.sample(5))
#     display(data.info())
#     display(data.describe(include='all'))

In [ ]:

# get_data_info(df)

In [ ]:
# features = df.drop(['price'], axis=1)
# target = df['price']

# X_train, X_test, y_train, y_test = train_test_split(
#     features, target, test_size=0.25, random_state=RANDOM_STATE
# )


In [ ]:
# # Создаем отложенную тестовую выборку

# X_test, features_test, y_test, target_test= train_test_split(
#     X_test, y_test, test_size=0.5, random_state=RANDOM_STATE
# )

In [ ]:
# get_data_info(X_train)

In [ ]:
# get_data_info(X_test)

In [ ]:
# #категориальные признаки для OHE Ridge
# ohe_features_ridge = X_train.select_dtypes(include='object').columns.to_list()
# print(ohe_features_ridge)

# #категориальные признаки для OHE RandomForestRegressor
# ohe_features_rf = ohe_features_ridge.copy()
# ohe_features_rf.remove('model')
# ohe_features_rf

In [ ]:
# #численные признаки
# #обратите внимание, что 'repaired' — категориальный бинарный признак.
# num_features = X_train.select_dtypes(exclude='object').columns.to_list()
# num_features.remove('repaired')
# num_features

In [ ]:
# models_train = set(X_train['model'].unique())
# models_test = set(X_test['model'].unique())
# num_models_train = len(models_train)
# num_models_test = len(models_test)
# print(f'''
# Количество уникальных значений признка "model" 
# в обеих выборках одинаковое: {num_models_train == num_models_test}
# ''')
# print(f'''
# Уникальные значения признка "model" 
# в обеих выборках одинаковые: {models_train == models_test}
# ''')
# print(f'''
# Только в тренировочной выборке есть значения: {models_train - models_test}
# ''')
# print(f'''
# Только в тестовой выборке есть значения: {models_test - models_train}
# ''')

In [ ]:
def time_score(model, features_train, target_train, features_test, target_test):
    
    start = time.time()    
    model.fit(features_train, target_train)
    train_time = time.time() - start
    
    start = time.time()  
    predict = model.predict(features_test)
    predict_time = time.time() - start

    return train_time, predict_time, mse(target_test, predict)**0.5

Выделим целевой признак. Разделим датасет:

In [ ]:
features = df.drop(['price'], axis = 1)  
target = df['price']

# features_train, features_valid, target_train, target_valid \
# =  train_test_split(features, target, test_size=0.25, random_state=41)

In [ ]:
# # Создаем отложенную тестовую выборку
# features_valid, features_test, target_valid, target_test \
# = train_test_split(features_valid, target_valid, test_size=0.5, random_state=41)

In [ ]:
# # Создаем объект OrdinalEncoder для кодирования категориальных признаков
# encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=np.int16)
# category_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']
# category_features
# encoder.fit(df[category_features])

# # Кодируем категориальные признаки
# df[category_features] = encoder.transform(df[category_features])
# df = df.astype(np.int16)
# df.info()

In [ ]:
# Создаем объект OrdinalEncoder для кодирования категориальных признаков
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=np.int16)
category_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']

# Разбиваем на train/valid/test
features_train, features_valid, target_train, target_valid \
=  train_test_split(features, target, test_size=0.25, random_state=41) # Создаем отложенную тестовую выборку
features_valid, features_test, target_valid, target_test \
= train_test_split(features_valid, target_valid, test_size=0.5, random_state=41)

# Кодируем категориальные признаки на train и используем ту же кодировку для valid и test
encoder.fit(features_train[category_features])
features_train[category_features] = encoder.transform(features_train[category_features])
features_valid[category_features] = encoder.transform(features_valid[category_features])
features_test[category_features] = encoder.transform(features_test[category_features])

# Проверяем тип данных
features_train = features_train.astype(np.int16)
features_valid = features_valid.astype(np.int16)
features_test = features_test.astype(np.int16)

In [ ]:
features_train.shape[0], features_valid.shape[0], target_train.shape[0], target_valid.shape[0], \
target_test.shape[0], features_test.shape[0]

In [ ]:
features_train['vehicle_type'] = features_train['vehicle_type'].astype('category')
features_train['gearbox'] = features_train['gearbox'].astype('category')
features_train['model'] = features_train['model'].astype('category')
features_train['fuel_type'] = features_train['fuel_type'].astype('category')
features_train['brand'] = features_train['brand'].astype('category')
features_train['repaired'] = features_train['repaired'].astype('category')

features_valid['vehicle_type'] = features_valid['vehicle_type'].astype('category')
features_valid['gearbox'] = features_valid['gearbox'].astype('category')
features_valid['model'] = features_valid['model'].astype('category')
features_valid['fuel_type'] = features_valid['fuel_type'].astype('category')
features_valid['brand'] = features_valid['brand'].astype('category')
features_valid['repaired'] = features_valid['repaired'].astype('category')

модель LightGBM

In [ ]:
%%time
estim = lgb.LGBMRegressor()
estim.fit(features_train, target_train, eval_metric='rmse', categorical_feature=category_features)
predicted_valid = estim.predict(features_valid)
predicted_train = estim.predict(features_train)
print('The rmse of LGBMRegressor prediction on train:', mse(target_train, predicted_train) ** 0.5)
print('The rmse of LGBMRegressor prediction on valid:', mse(target_valid, predicted_valid) ** 0.5)

Разброс значений цены между обучающей и валидационной выборках маленький.

In [ ]:
%%time
model = lgb.LGBMRegressor(num_leaves= 31, learning_rate=0.01, max_depth=15, n_estimators=250)
model.fit(features_train, target_train, categorical_feature=category_features)
predicted_valid1 = model.predict(features_valid)
rmse = (mse(target_valid, predicted_valid1))**0.5

rmse

In [ ]:
result = []
result.append(time_score(model, features_train, target_train, features_valid, target_valid))

Градиентный бустинг CatBoost

In [ ]:
%%time
model_cat = CatBoostRegressor(n_estimators = 200, loss_function = 'MAE', eval_metric = 'RMSE'
                              , cat_features = category_features)
model_cat.fit(features_train, target_train, cat_features=category_features, verbose=10)
predicted_valid2 = model_cat.predict(features_valid)
rmse = (mse(target_valid, predicted_valid2))**0.5
rmse

In [ ]:
result.append(time_score(model_cat, features_train, target_train, features_valid, target_valid))

In [ ]:
pd.DataFrame(data=result,
index=['LightGBM', 'CatBoost'],
columns=['Время_обучения', 'Время_Предсказания', 'RMSE'])

Теперь перейдем к моделям не являющимся бустинговыми. Возьмем случайный лес и дерево решений. Попробуем для них другой подход по обучению.

Случайный лес

In [ ]:
%%time
# Создаем модель
forest_model = RandomForestRegressor(random_state=12345, n_estimators=100)

# Обучаем модель на обучающей выборке
forest_model.fit(features_train, target_train)

# Получаем предсказания для валидационной выборки
predictions = forest_model.predict(features_valid)

# Вычисляем RMSE для валидационной выборки
rmse = mse(target_valid, predictions)**0.5
print('RMSE на валидационной выборке:', rmse)


Решающее дерево

In [ ]:
%%time
# создаем объект решающего дерева
tree_model = DecisionTreeRegressor(max_depth=10, random_state=42)

# обучаем модель на тренировочном наборе данных
tree_model.fit(features_train, target_train)

# получаем предсказания на валидационном наборе данных
tree_predictions = tree_model.predict(features_valid)

# оцениваем качество модели
tree_rmse = mse(target_valid, tree_predictions)**0.5
print('RMSE решающего дерева:', tree_rmse)

In [ ]:
# Создаем пустой DataFrame с нужными колонками
result = pd.DataFrame(columns=['Время_обучения', 'Время_Предсказания', 'RMSE'])

# Добавляем результаты для дерева решений
result.loc['DecisionTree'] = time_score(tree_model, features_train, target_train, features_valid, target_valid)

# Добавляем результаты для случайного леса
result.loc['RandomForest'] = time_score(forest_model, features_train, target_train, features_valid, target_valid)

# Выводим результаты в табличном виде
print(result)

Решающее дерево выигрывает по скорости у случайного леса хоть и RMSE выше. Но поскольку оно подходит по условию задания в преоритете оставим время. Теперь найдем более подходящие пораметры.

In [ ]:
%%time
# Задаем сетку параметров для перебора
param_grid = {'max_depth': [5, 10, 20, None],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              'max_features': ['auto', 'sqrt', 'log2']}

# Создаем модель
model = DecisionTreeRegressor(random_state=12345)

# Используем кросс-валидацию для подбора параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(features_train, target_train)

# Выводим лучшие параметры
print('Лучшие параметры:', grid_search.best_params_)

# Получаем предсказания для валидационной выборки с помощью лучшей модели
best_model = grid_search.best_estimator_
predicted_valid = best_model.predict(features_valid)

# Оцениваем качество модели на валидационной выборке
mse_valid = mse(target_valid, predicted_valid)
rmse_valid = mse_valid ** 0.5
print('RMSE на валидационной выборке:', rmse_valid)

In [ ]:
%%time
# Получаем предсказания для тестовой выборки с помощью лучшей модели
predicted_test = best_model.predict(features_test)

# Оцениваем качество модели на тестовой выборке
mse_test = mse(target_test, predicted_test)
rmse_test = mse_test ** 0.5
print('RMSE на тестовой выборке:', rmse_test)

Линейная регресия

In [ ]:
# # создание объекта модели
# model = LinearRegression()

# # обучение модели на тренировочной выборке
# model.fit(features_train, target_train)

# # оценка качества модели на отложенной выборке
# predict = model.predict(features_valid)
# rmse = mse(target_valid, predict, squared=False)
# print('RMSE на отложенной выборке:', rmse)

In [ ]:
# # Определим random_state
# RANDOM_STATE=41

In [ ]:
# # выделим категориальные признаки

# category_features = ['vehicle_type', 'gearbox', 'brand', 'model', 'fuel_type']

# # выделим определим числовые признаки
# number_features = ['registration_year', 'power', 'kilometer', 'repaired']

# # Оставим в DF только то, что нужно (с обработкой категориальных признаков)
# df = df[number_features + category_features + ['price']];
# df_simple = df.copy()
# df_simple.head()

In [ ]:
# # Подготовим наши категориальные признаки, будем использовать технику OHE
# features_ohe = pd.get_dummies(features, columns=category_features, drop_first=True)
# features_ohe.head()

In [ ]:
# # Проведем скалирование числовых признаков
# scaler = StandardScaler()

# scaler.fit(features_ohe[number_features])
# features_ohe_scaled = features_ohe.copy()
# features_ohe_scaled[number_features] = scaler.transform(features_ohe_scaled[number_features])
# features_ohe_scaled.head()

In [ ]:
# # Поделим нашу выборку в пропорции 75-25, т.к. параметры мы будем искать с помощью кросс-валидации через GridSearchCV

# # Сначала разделим на валидационную и обучающую выборку.

# def create_train_valid_and_test_set(features, target):
#     features_train, df_valid_ant_test_features, target_train, df_valid_ant_test_target = train_test_split(
#         features, target, test_size=0.4, random_state=RANDOM_STATE)

#     # теперь разделим тестовую выборку от валидационной 
#     features_valid, features_test, target_valid, target_test = train_test_split(
#         df_valid_ant_test_features, df_valid_ant_test_target, test_size=0.5, random_state=RANDOM_STATE)

#     del df_valid_ant_test_features, df_valid_ant_test_target
    
#     # Проверим, что получилось
#     print('Target test count = {}, Valid count = {} \
#           , Train test count = {}'.format(target_test.count(), target_valid.count(), target_train.count()))
    
#     return features_train, target_train, features_valid, target_valid, features_test, target_test

# def create_train_and_test_set(features, target):
#     features_train, features_test, target_train, target_test = train_test_split(
#         features, target, test_size=0.25, random_state=RANDOM_STATE)
    
#     # Проверим, что получилось
#     print('Target test count = {}, Train test count = {}'.format(target_test.count(), target_train.count()))
    
#     return features_train, target_train, features_test, target_test

# features_train, target_train, features_test, target_test = create_train_and_test_set(features_ohe_scaled, target)

In [ ]:
# # Определим функцию для подсчета RMSE
# def rmse(target, predicted):
#     return mse(target, predicted) ** 0.5

In [ ]:
# %%time

# # Попробуем обучить случайный лес
# parameters = {'n_estimators': range(5,11,5),'max_depth': range(1,8,3), 'min_samples_leaf': range(1,5,2)
#               , 'min_samples_split': range(2,6,2)}

# model = RandomForestRegressor()
# grid = GridSearchCV(model, parameters, cv=3, scoring='neg_mean_squared_error')
# grid.fit(features_train, target_train)

# grid.best_params_


In [ ]:
# # Чтобы не собирать вручную результаты, напишем класс, который будет сохранять время обучения, время предсказания и RMSE

# class ModelHelper:
#     def __init__(self, name, model):
#         """Constructor"""
#         self.name = name
#         self.model = model
    
#     def fit(self, features, target):
#         start = time.time()    
#         self.model.fit(features, target)
#         end = time.time() - start
#         self.fitTime = end
    
#     def predict(self, features, target):
#         start = time.time()  
#         predicted = self.model.predict(features)
#         end = time.time() - start
#         self.predictedTime = end
#         self.rmse = rmse(target, predicted)
    
#     def getResult(self):
#         return self.name, self.fitTime, self.predictedTime, self.rmse

In [ ]:
# # Создадим переменную, где будут храниться результаты
    
# result = []

In [ ]:
# #И создадим функцию для вывода по красоте
# def print_result(result):
#     print('|{: >20}|{: >18}|{: >18}|{: >12}|'.format('Модель', 'Время обучения', 'Время предсказания', 'RMSE'))
#     for info in result:
#         print('|{: >20}|{: >18.5f}|{: >18.5f}|{: >12.5f}|'.format(info[0], info[1], info[2], info[3]))

Решающее дерево

In [ ]:
# # Проверим решающее дерево
# model_tree = DecisionTreeRegressor(max_depth=7, random_state=RANDOM_STATE)
# model_tree_helper = ModelHelper('Random Tree', model_tree)
# model_tree_helper.fit(features_train, target_train)
# model_tree_helper.predict(features_test, target_test)
# result.append(model_tree_helper.getResult())

Случайный лес

In [ ]:
# # Проверим наш случайный лес
# model_forest = RandomForestRegressor(max_depth=7, min_samples_leaf=3
#                                      , min_samples_split=2, n_estimators=10, random_state=RANDOM_STATE)
# model_forest_helper = ModelHelper('Random forest', model_forest)
# model_forest_helper.fit(features_train, target_train)
# model_forest_helper.predict(features_test, target_test)
# result.append(model_forest_helper.getResult())

Линейная регрессия

In [ ]:
# # Попробуем линейную регрессию
# model_lr = LinearRegression()
# model_lr_helper = ModelHelper('Linear regression', model_lr)
# model_lr_helper.fit(features_train, target_train)
# model_lr_helper.predict(features_test, target_test)
# result.append(model_lr_helper.getResult())

In [ ]:
# # Посмотрим на промежуточный результат
# print_result(result)

Резюме.

При обучении модели и проверки ее скорости было выбрано 4 варианта. Градиентный бустинг LightGBM, CatBoost и простые модели  случайный лес, дерево решений. Было решено сделать для бустинга и простых моделей немного разные подходы к написанию кода. 

## Анализ моделей
<a id='section_id3'></a>

При проведении получения предсказаний модели проявили себя следующим образом:


* RandomForestRegressor - RMSE 1592, time 1min 23s
* DecisionTreeRegressor - RMSE 1964, time 634ms

* CatBoost - RMSE 1823, time 1 min
* LightGBM - RMSE 1880, time 30 min

По величине RMSE все модели удовлетворяют требуемому условию (RMSE < 2500). Остальные условия так жестко не определены поэтому точно небудем рассматривать модель которая превышает этот пораметр, у других моделий будем смотреть больше на время чем на RMSE, для выбора наиболее подходящий заказчику.

По скорости получения предсказаний модели выигрывает Random Tree. Опираясь на критерии заказчика, выберав Random Tree, проверяем её качество на тестовой выборке. Для максимального удобства заказчика лучшую модель расписали полностью, (чтобы заказчику неприходилось запускать "2. Обучение модели".

In [ ]:

# # Создаем объект OrdinalEncoder для кодирования категориальных признаков
# encoder = OrdinalEncoder()

# # Кодируем категориальные признаки
# cat_features = ['vehicle_type', 'gearbox', 'fuel_type', 'brand', 'repaired', 'model']
# df[cat_features] = encoder.fit_transform(df[cat_features])

In [ ]:
# # Сначала разделим на валидационную и обучающую выборку.
# # отделим 40% данных для валидационной выборки
# df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
# # извлекаем признаки 
# features = df_valid.drop('price', axis=1) # извлекаем признаки 
# target = df_valid['price']# извлекаем целевой признак

In [ ]:
# # Теперь отделим 50% данных от валидационной выборки
# features_test, features_valid, target_test, target_valid = train_test_split(
#     features, target, test_size=0.5, random_state=12345)
# features_train = df_train.drop('price', axis=1)
# target_train = df_train['price']

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.metrics import mean_squared_error

# # Создаем модель и обучаем ее на обучающей выборке
# model = DecisionTreeRegressor(random_state=12345)
# model.fit(features_train, target_train)

# # Получаем предсказания для валидационной выборки
# predicted_valid = model.predict(features_valid)

# # Оцениваем качество модели на валидационной выборке
# mse_valid = mse(target_valid, predicted_valid)
# rmse_valid = mse_valid ** 0.5
# print('RMSE на валидационной выборке:', rmse_valid)

In [ ]:
# %%time
# # Задаем сетку параметров для перебора
# param_grid = {'max_depth': [5, 10, 20, None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'max_features': ['auto', 'sqrt', 'log2']}

# # Создаем модель
# model = DecisionTreeRegressor(random_state=12345)

# # Используем кросс-валидацию для подбора параметров
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(features_train, target_train)

# # Выводим лучшие параметры
# print('Лучшие параметры:', grid_search.best_params_)

# # Получаем предсказания для валидационной выборки с помощью лучшей модели
# best_model = grid_search.best_estimator_
# predicted_valid = best_model.predict(features_valid)

# # Оцениваем качество модели на валидационной выборке
# mse_valid = mse(target_valid, predicted_valid)
# rmse_valid = mse_valid ** 0.5
# print('RMSE на валидационной выборке:', rmse_valid)

In [ ]:
# %%time
# # Получаем предсказания для тестовой выборки с помощью лучшей модели
# predicted_test = best_model.predict(features_test)

# # Оцениваем качество модели на тестовой выборке
# mse_test = mse(target_test, predicted_test)
# rmse_test = mse_test ** 0.5
# print('RMSE на тестовой выборке:', rmse_test)

Резюме.

По скорости получения предсказаний модели выигрывает Random Tree. DecisionTreeRegressor на тестовой выборке: 1810 time 15.9 ms.

# Вывод
<a id='section_id4'></a>

Для сервиса по продаже автомобилей с пробегом «Не бит, не крашен» построили модель, которая поможет определять рыночную стоимость своего автомобиля. В нашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей. Критерии, которые важны заказчику (качество предсказания, время обучения модели, время предсказания модели). при этом значение метрики RMSE должно быть меньше 2500.

Загрузили и изучили данные.Удалили явные дублекаты. Удаляем неинформативные признаки 'DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'. Приведем названия для дальнейшей работы к змеинному иедексу. Работаем над аномалиями. Год мы берем с 1913 первый автомобильный канвеер по 2023 (текущий). Проверили месяц на кореляцию с ценой, результат вышел небольшим, удалили этот признак. Было решено удалить машины с мощьностью более 388 л.с. и был использвать перебор для машин мощьностью менее 26 л.с. им присвоины мощьности соответствующие авто с совпадающим годом моделью и маркой автомобиля. С ценой была проделана аналогичная операция в диапозоне менее 500 евра. Машины на которые неподошли похожие были удалены.
Данные по присутствию или отсутствию ремонта мы перевели в цифровой вид 0 будет отсутствие ремонта, 1 неизвестно и 2 - ремонт был. Остальные пропуски заменили категорией undefined. Признаки которые могли перевели в int. Было удалено менее 4% информации с датасета.

При обучении модели и проверки ее скорости было выбрано 4 варианта. Градиентный бустинг LightGBM, CatBoost и простые модели случайный лес, дерево решений.

При проведении получения предсказаний модели проявили себя следующим образом:

* RandomForestRegressor - RMSE 1592, time 1min 23s
* DecisionTreeRegressor - RMSE 1964, time 634ms

* CatBoost - RMSE 1823, time 1 min
* LightGBM - RMSE 1880, time 30 min

Опираясь на критерии заказчика, выберав Random Tree, проверяем её качество на тестовой выборке.